In [21]:
import time
from ont_fast5_api.fast5_interface import get_fast5_file
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
import math
import h5py
import operator
import json

In [2]:
def loadReads(f5_file="../../similar_testdata/similar_squiggles.fast5", withMetadata=False):
    '''
        returns list of numpy arrays representing reads as well as dataframe containing reads metadata.
    '''
    print("reading squiggles file...")
    start = time.time()
    reads = []
    if withMetadata:
        ids = []
        length = []
        with get_fast5_file(f5_file, mode="r") as f5:
            for read in f5.get_reads():
                raw_data = read.get_raw_data()
                reads.append(raw_data)
                ids.append(read.read_id)
                length.append(len(raw_data))

        z = list(zip(ids, length))
        print(f"Took {time.time()-start} seconds.\n")
        return reads, pd.DataFrame(z, columns=["id", "length"])

    else:
        with get_fast5_file(f5_file, mode="r") as f5:
            for read in f5.get_reads():
                raw_data = read.get_raw_data()
                reads.append(raw_data)
        print(f"Took {time.time()-start} seconds.\n")
        return reads

In [3]:
def removeSpikes(sequence):
    # print("removing spikes")
    # start = time.time()
    
    spikeThresh = 2000
    returnList = [i for i in sequence if abs(i) < spikeThresh]
    # print(f"Took {time.time()-start} seconds.\n")

    return returnList

In [4]:
def zScoreNormalise(sequence):
    # print("z-score normalising")
    # start = time.time()
    
    zScoreNormalised = np.zeros(len(sequence))
    mean = np.mean(sequence)
    standardDev = np.std(sequence)
    zScoreNormalised = (sequence - mean) / standardDev
    # print(f"Took {time.time()-start} seconds.\n")

    return zScoreNormalised

In [5]:
def reduceResolution(sequence):
    #resolution reduction method based on significant changes in values
    # print("reducing resolution")
    # start = time.time()

    thresh = 0.2

    differences = []

    for i in range(len(sequence)-1):
        differences.append(abs(sequence[i+1] - sequence[i]))

    differences.insert(0, 0)		#inserting 0 at 0th index to compensate for shift

    resReducedSquiggle = []
    sig = 0
    for i in range(len(sequence)):
        if differences[i] > thresh:
            sig = sequence[i]
        resReducedSquiggle.append(sig)

    # print(f"Took {time.time()-start} seconds.\n")
    
    return resReducedSquiggle

In [6]:
def squeezeTime(sequence):
    # print("squeezing time")
    # start = time.time()
    
    timeNormalised = []
    for i in range(1, len(sequence)):
        if sequence[i] != sequence[i-1]:
            timeNormalised.append(sequence[i])

    # print(f"Took {time.time()-start} seconds.\n\n\n")
    return timeNormalised

In [7]:
def fullNormalise(sequence):
    '''
    1) spike removal
    2) z-score normalisation
    3) resolution reduction
    4) time squeezing
    '''
    s = sequence.copy()
    #print(f"seuqnce length: {len(s)}")

    s = removeSpikes(s)
    #print(f"removed spikes length: {len(s)}")

    s = zScoreNormalise(s)
    #print(f"zScore normal length: {len(s)}")

    s = reduceResolution(s)
    #print(f"resreduced length: {len(s)}")

    s = squeezeTime(s)
    #print(f"time squeezed length: {len(s)}")

    return s

In [8]:
def normaliseDataset(squiggles, path="../../normalised/"):
    batchSize = 4000#200
    lastBatch = False
    batchNo = 0
    totalSquiggleCount = len(squiggles)
    totalBatches = math.floor(totalSquiggleCount/batchSize)
    start = time.time()
    while not lastBatch:
        print(f"Batch {batchNo} / {totalBatches}")
        if batchNo == totalBatches:
            lastBatch = True
            batch = squiggles[batchNo*batchSize:]
        else:
            batch = squiggles[batchNo*batchSize:(batchNo+1)*batchSize]

        
        f = h5py.File(f"{path}normalised-batch{batchNo}.hdf5", "a")
        dt = h5py.vlen_dtype(np.dtype("float16"))                           #non-normalised data stored as int16 so only need 16 bits of precision.
        vectorsDset = f.create_dataset(f"vectors{batchNo}", (len(batch),), dtype=dt)

        for i in range(len(batch)):
            normalised = fullNormalise(batch[i])
            vectorsDset[i] = normalised
            print(f"(Batch {batchNo}) - squiggle {i+(batchNo*batchSize)} / {totalSquiggleCount}")

        f.close()

        batchNo += 1
    print(f"Took {time.time()-start} seconds.\n")


In [18]:
def getSimilarity(squig1, squig2):
    sim = np.correlate(squig1, squig2)
    return(np.max(sim))

In [9]:
allSquiggles, metadata = loadReads(withMetadata=True)

reading squiggles file...
Took 7.36430811882019 seconds.



NB The slowest section by far is the "reduceResolution" function - optimisations should be first focussed on simplifying this function.

In [10]:
#normalSquiggles = [fullNormalise(i) for i in allSquiggles]

normaliseDataset(allSquiggles)

Batch 0 / 0
(Batch 0) - squiggle 0 / 3760
(Batch 0) - squiggle 1 / 3760
(Batch 0) - squiggle 2 / 3760
(Batch 0) - squiggle 3 / 3760
(Batch 0) - squiggle 4 / 3760
(Batch 0) - squiggle 5 / 3760
(Batch 0) - squiggle 6 / 3760
(Batch 0) - squiggle 7 / 3760
(Batch 0) - squiggle 8 / 3760
(Batch 0) - squiggle 9 / 3760
(Batch 0) - squiggle 10 / 3760
(Batch 0) - squiggle 11 / 3760
(Batch 0) - squiggle 12 / 3760
(Batch 0) - squiggle 13 / 3760
(Batch 0) - squiggle 14 / 3760
(Batch 0) - squiggle 15 / 3760
(Batch 0) - squiggle 16 / 3760
(Batch 0) - squiggle 17 / 3760
(Batch 0) - squiggle 18 / 3760
(Batch 0) - squiggle 19 / 3760
(Batch 0) - squiggle 20 / 3760
(Batch 0) - squiggle 21 / 3760
(Batch 0) - squiggle 22 / 3760
(Batch 0) - squiggle 23 / 3760
(Batch 0) - squiggle 24 / 3760
(Batch 0) - squiggle 25 / 3760
(Batch 0) - squiggle 26 / 3760
(Batch 0) - squiggle 27 / 3760
(Batch 0) - squiggle 28 / 3760
(Batch 0) - squiggle 29 / 3760
(Batch 0) - squiggle 30 / 3760
(Batch 0) - squiggle 31 / 3760
(Batch

In [22]:
#for each squiggle:
#   compare to every other squiggle
#   make a note of the maximum similarity found from cross correlation with every other squiggle
#   return dictionary of squiggle: similarity score
#   store the top 10 for each squiggle

f = h5py.File("../../normalised/normalised-batch0.hdf5", "r")
savePath = "../../correlation_compared/"

data = f["vectors0"]
ids = metadata["id"].tolist()
#print(metadata["id"].tolist())
print(len(ids))
print(len(data))

similarities = {}
for i in range(len(ids)):
    print(f"Dataset sweep for squiggle {ids[i]} ({i}/{len(ids)})")
    similarities[ids[i]] = {}
    for j in range(len(ids)):
        print(j)
        similarities[ids[i]][ids[j]] = getSimilarity(data[i], data[j])
        
    s = dict(sorted(similarities[ids[i]].items(), key=operator.itemgetter(1), reverse=True)[:10])     #saves just the top 10 most similar
    with open(f"{savePath}{ids[i]}.json", "w") as jsonFile:
        jsonFile.write(json.dumps(s))

with open(f"{savePath}00_allSquiggles.json", "w") as jsonFile:
    jsonFile.write(json.dumps(similarities))



3760
3760
Dataset sweep for squiggle 00014eb4-4e2c-4087-ac24-57dea735b7b4 (0/3760)
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256

TypeError: Object of type float16 is not JSON serializable